In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('perfume.csv')

Basic data overview using pandas

In [2]:
data.shape
# data.drop() #columns
# data.nlargest()
# data.isnull().sum()
# data.url.unique()
# data.nunique(axis=0)
# brand_data = data.groupby('brand').count()
# brand_data.sort_values(by='brand')
# data.Tags.value_counts()
# max, mean, median, min, mode, quantile, rank, round, sum, std, var
# Df = df[x < 50] filtering 

(968, 16)

In [ ]:
data.columns

In [ ]:
data.info()

In [8]:
data = data.drop('Unnamed: 14', axis = 1)

In [9]:
# data.brand = data.brand.astype(str)
# separated the perfumes from the sample packs into 2 separate dataframes
# simple check that the numbers add up 
samplepack_df = data[data.brand.str.contains('Pack')==True]
print(samplepack_df.shape)

perfume_df = data[data.brand.str.contains('Pack')==False]
# perfume_df = data[data['brand'].notna() & ~data['brand'].str.contains('Pack')]
# perfume_df = data[data['brand'].str.contains('Pack', na=False) == False]
print(perfume_df.shape)

perfume_df
# pdf = data.where(data.brand.str.contains('Sample Packs')==True) only for whole object

(32, 15)
(935, 15)


,url,href,name,brand,description,category,badges,prices,topnotes,midnotes,basenotes,Notes,Tags,Style,similar_perfumes
0,https://bloomperfume.co.uk/products/salted-gre...,salted-green-mango,Salted Green Mango,Strangers Parfumerie,Bao Mangos and Childhood,NaN,"['Bestseller', 'Exclusive']","['£80.00, 30 ml EdP', '£4.00, 1 ml EdP', '6 to...",NaN,NaN,NaN,"['Guava Leaves', 'Vetiver', 'Chili', 'Bigarade...",['#Fresh'],"['For Him', 'Unisex', 'For Her']","['antigua', 'manguier-metisse', 'oliver', 'sun..."
1,https://bloomperfume.co.uk/products/memoirs-of...,memoirs-of-a-trespasser,Memoirs of a Trespasser,Imaginary Authors,Smoky Vanilla,"['Edible', 'Woody']",['Bestseller'],"['£90.00, 50 ml EdP', '£3.00, 1 ml EdP', '4 to...",NaN,NaN,NaN,"['Madagascar Vanilla', 'Ambrette Seeds', 'Cogn...","['#Sweet', '#Sexy']","['For Him', 'Unisex', 'For Her']","['vanilla-shot', 'anima-dulcis', 'vi-et-armis'..."
2,https://bloomperfume.co.uk/products/falling-in...,falling-into-the-sea,Falling into the Sea,Imaginary Authors,Salty sunny and zesty,NaN,['Bestseller'],"['£90.00, 50 ml EdP', '£3.00, 1 ml EdP', '4 to...",NaN,NaN,NaN,"['Sea Salt', 'Sea Spray', 'Bergamot', 'Hot San...",['#Sexy'],"['For Him', 'Unisex', 'For Her']","['arancia-di-sicilia', 'still-life-in-rio', 'c..."
3,https://bloomperfume.co.uk/products/t-rex,t-rex,T-Rex,Zoologist,Indolic Smoky Floral,NaN,"['Bestseller', 'Exclusive']","['£210.00, 60 ml Extrait de Parfum', '£7.00, 1...","['Black Pepper', 'Fir Balsam', 'Bergamot', 'Nu...","['Osmanthus', 'Ylang-Ylang', 'Champaca', 'Smok...","['Madagascar Vanilla', 'Frankincense', 'Cade',...",NaN,"['#Animalic', '#Smoky', '#Beast']","['For Him', 'Unisex', 'For Her']","['a-city-on-fire', 'chetyre', 'oud-imperial-ex..."
4,https://bloomperfume.co.uk/products/bee,bee,Bee,Zoologist,Dense Sweet Leather,"['Edible', 'Leather', 'Woody']","['Bestseller', 'Exclusive']","['£210.00, 60 ml Extrait de Parfum', '£7.00, 1...","['Candied Orange', 'Royal Jelly Accord', 'Ging...","['Broom', 'Honey', 'Mimosa', 'Heliotrope', 'Or...","['Sandalwood', 'Benzoin', 'Labdanum', 'Tonka B...",NaN,"['#Animalic', '#Sweet', '#Beast']","['For Him', 'Unisex', 'For Her']","['choeur-des-anges', 'opulent-wood', 'or-du-sr..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,https://bloomperfume.co.uk/products/garofano,garofano,Garofano,Lorenzo Villoresi,NaN,"['Floral', 'Spicy', 'Vintage/Old School']",['New'],"['£135.00, 100 ml EdT', '£4.00, 1 ml EdT', '3 ...",NaN,NaN,NaN,"['Spices', 'Rose', 'Carnation', 'Geranium']","['#Fresh', '#Sweet']",['Unisex'],"['sweet-william', '500-years', 'sleight-of-fer..."
964,https://bloomperfume.co.uk/products/perfume-se...,perfume-set-4-18-ml,Perfume set,Aura of Kazakhstan,NaN,"['Aromatic', 'Fruity', 'Unconventional', 'Woody']","['Exclusive', 'New']","['£295.00, 4x18 ml EdP']",NaN,NaN,NaN,"['Crude Oil', 'Vetiver', 'Apple', 'Nutmeg', 'H...","['#Sweet', '#Sexy', '#Smoky', '#Beast']",['Unisex'],"['travel-set', 'short-stories-set', '4-seasons..."
965,https://bloomperfume.co.uk/products/tonkamande,tonkamande,Tonkamande (Discontinued),Pierre Guillaume - Parfumerie Générale,Balmy Aldehydes,"['Edible', 'Transparent']",[],"['£117.50, 100 ml EdP', '£81.50, 50 ml EdP', '...",['Almond'],['Aldehydes'],"['Tonka Beans', 'Wheat', 'Amber']",NaN,NaN,['For Her'],"['lumire-blanche', 'tardes', 'joking-aside', '..."
966,https://bloomperfume.co.uk/products/corps-et-ames,corps-et-ames,Corps et Ames (Discontinued),Pierre Guillaume - Parfumerie Générale,Feminine Chypre,"['Green', 'Transparent']",[],"['£117.50, 100 ml EdP', '£81.50, 50 ml EdP', '...","['Verbena', 'Geranium']","['Jasmine Sambac', 'Patchouli']","['Sandalwood', 'Musk']",NaN,NaN,['For Her'],"['pg-corps-et-ames-edt-apaisante', 'ella', 'ma..."


The following code is to clean the 'prices' column into separate columns for perfume strength/formula and separate prices for different volume bottles on sale

In [58]:
# split the prices column by the string 'ml' and create a new dataframe to hold these
prices_list = perfume_df.prices.str.split('ml')
# reversed the order of the lists so that the strength and the smallest column come first
prices_list = prices_list.apply(lambda x: list(reversed(x)))
# created a new dataframe with the split list 
df_prices = pd.DataFrame(prices_list.tolist()).fillna('').add_prefix('price_')
df_prices = df_prices.drop(['price_1', 'price_2'], axis = 1)


df_prices.columns
df_prices = df_prices.rename(columns={"price_0": "formula", "price_3": "price_1", "price_4": "price_2", "price_5":"price_3"})

df_prices = df_prices.replace(r'£|\[|\]|\'|\"|\?|EdP Whats this Learn more|roll-on|EdP\', |EdT\', |Extrait de |Eau de Cologne', '', regex=True)
df_prices.price_1 = df_prices.price_1.replace('Parfum,','', regex=True)
df_prices = df_prices.replace('Eau de Cologne', 'EdC', regex=True)
df_prices = df_prices.apply(lambda x: x.str.strip())

df_1 = df_prices.price_1.str.split(pat=', ', expand=True)
df_2 = df_prices.price_2.str.split(pat=', ', expand=True)
df_3 = df_prices.price_3.str.split(pat=', ', expand=True)
up_todate = pd.concat([df_prices, df_1, df_2, df_3], axis=1)
print(up_todate.columns)

new_columns = ['formula', 'price_1', 'price_2', 'price_3','p_1', 'v_1', 'p_2', 'v_2', 'p_3', 'v_3']
up_todate.columns = new_columns

# used the following to check for missed regex phrases and odd columns 
# print(up_todate.unsure.value_counts())
# values = up_todate.unsure.to_list()
# indices = [index for index, value in enumerate(values) if value == '15']
# print(indices)

# up_todate.loc[934] # remove EdP Whats this Learn more ????
# update readme 

Index(['formula', 'price_1', 'price_2', 'price_3', 0, 1, 0, 1, 0, 1], dtype='object')


formula    EdP Whats this Learn more
price_1                    81.50, 50
price_2                  117.50, 100
price_3                             
p_1                            81.50
v_1                               50
p_2                           117.50
v_2                              100
p_3                                 
v_3                             None
Name: 934, dtype: object

In [ ]:
# group by brand 
# some basic summary things 
# group by gender
# group by bestseller
# group by exclusive
